# **Sosyal Medya Verisi İşleme**

---


* *Mentör* 
 * **Umut Rezan Azizoğlu**

* *Grup Üyeleri* 
 * **Gökhan Atasoy** 
 * **Caner Koyun**
 * **Ebru Dalkır** 
 * **Gizem Arslan**
 * **Halenur Kar**
 * **Mine Yaman**
 * **Oğuzhan Turan**

---

* *PROBLEM*

* Kadıköy Belediyesinin de dahil olduğu; Marmara Belediyeler Birliği’ne üye 93 belediye ile yapılan; belediyelerin sosyal medya araçlarından ne düzeyde yararlandığını ve bu araçların hangi amaçlar doğrultusunda kullandığını analiz eden; çalışmaya göre;
 * Hâlihazırda, belediyelerin sadece %37,1’i kurumsal sosyal medya politikasına sahiptir.
 * Belediyelerin sadece %8’i yazılı bir sosyal medya rehberine sahiptir.
 * Belediyelerden %77,5’i vatandaşla iletişim için kurumsal sosyal medya hesaplarını kullanma yetkisini birkaç kişi ilesınırlı tutmaktadır.
 * Belediyelerden %58,7’si personelinin işyerinde sosyal medya araçlarına erişimine imkân vermektedir.
 * Belediyelerden sadece %6,7’si personelinin kişisel sosyal medya kullanımını düzenleyen bir politikaya sahiptir.
 * Belediyelerden %30,4’ü personeline sosyal medya eğitimi vermektedir.
 * Belediyelerden sadece %5,5’i meclis üyelerine sosyalmedya eğitimi vermektedir.
 * Proje için yaptığımız çalışmaya göre Kadıköy belediyesinde iki vardiya halinde iki kişi görev almaktadır.


---

* *Çalışma Özeti*

 * Bununla bağlantılı olarak Kadıköy Belediyesinin sosyal medya platformlarından olan, Twitter’da; Kadıköy ve Kadıköy Belediyesi hakkında ki tweetlere geri dönüş oranı %14’tür.
 * Kadıköy Belediyesinin Sosyal Medya konusundaki organizasyonunu desteklemek ve sosyal medyadan daha fazla verim almasını sağlamak üzere, çalışmamızı saptadığımız problemler üzerinden şekillendirdik.
 * Öncelikli hedef olarak Sosyal Medya platformlarından Twitter’ı odak noktası seçtik. Ve içeriğinde “Kadıköy belediyesi” ve “Kadıköy” geçen, 19 -26 Ağustos tarihindeki tüm twitler ile çalışmaya başladık.
 * Çalışmamızda NLP (Natural Language Processing) – (Doğal Dil İşleme) tekniklerinden yararlanarak, veriyi bilgisayarlar için anlaşılır, hale getirdik.
 * Çalışmayı bir kategorileştirme problemi olarak gördük ve gelen tüm twitleri 4 kategoriden biri olarak olarak tahminlemesi için eğittik.
 * Kategorilerimizi; “Bilgilendirme”, “Talep”, “Selamlama” ve “Diğer” Twitler şeklinde analizlerimiz sonucunda belirledik.
 * İşlemden geçen veriler ile Makine öğrenmesi algoritmalarından olan Lojistik regresyon ve  Naive Bayesi kullandık. Ek olarakta Derin öğrenme yöntemlerinden olan RNN yöntemini kullanarak, günlük hayatta uygulanabilir yapay zeka  modelleri ortaya çıkardık.






# **1. Adım**:  **Verinin oluşturulması**

---

Verileri Twitter üzerinden Twitter API kullanarak çektik

### Notebook'un Drive'a Mount Edilmesi

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
import os

os.chdir('My Drive')
os.chdir('Colab Notebooks')

### Gerekli Kütüphanelerin Eklenmesi

Twitter üzerinden veri çekebilmemiziçin gerekli olan kütüphaneyi ekledik.

In [0]:
pip install tweepy==3.6.0

In [0]:
import tweepy
from tweepy import OAuthHandler
import pandas as pd
import json
from collections import OrderedDict

### Twitter API'na Bağlanılması

In [0]:
consumer_key = '###########'
consumer_secret = '################'
access_token = '################'
access_secret = '#################'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True) # "wait_on_rate_limit = True" olarak tanımlayarak veri çekme kısıtlamasının önüne geçiyoruz.

### Kimlik Sorgusu

Twitter API'na bağlandıktan sonra kimliğimizi sorguluyoruz.

In [0]:
user = api.me()
print('Name: ' + user.name)
print('Description: ' + user.description)
print('Friends: ' + str(user.friends_count))
print('Nick: @' + str(user.screen_name))

### Verinin Çekilmesi 

Twitter üzerinden son bir haftada (19 - 26 Ağustos 2019) 'kadıköy belediyesi' ve '@kadikoybelediye' geçen tweetleri çekip dataframe'e yazdırdık.

In [0]:
tweetid = []
tweets = []
ttime = []
hashtags = []
nick = []
location = []

for status in tweepy.Cursor(api.search, q=('kadikoy' + 'belediye' or '@kadikoybelediye'), tweet_mode="extended").items(5000):
  tweetid.append(status.id_str)
  tweets.append(status.full_text.lower() )
  ttime.append(status.created_at)
  hashtags.append (status._json["entities"]["hashtags"][0]["text"]   if len(status._json["entities"]["hashtags"])> 0 else None)
  nick.append(status.user.screen_name)
  location.append(status.user.location)
  
Tweetdfread = pd.DataFrame ({"tweetid" : tweetid,
                         'tweets' : tweets,
                         'ttime' : ttime,
                         'hashtags' : hashtags,
                         'nick' : nick,
                         'location' : location
    
})


### Verinin Kaydedilmesi

Oluşturduğumuz dataframe'i csv formatında çalıştığımız klasöre kaydettik.

In [0]:
Tweetdfread.to_csv( 'KBTweet.csv',sep=';', index = False)

# **2. Adım**:  **Verinin temizlenmesi**

---



### Gerekli Kütüphanelerin Eklenmesi
 
Stemming aşamasında kullanmak üzere TurkishStemmer kütüphanesini ortama yükledik.

In [0]:
pip install TurkishStemmer

In [0]:
import numpy as np

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

nltk.download('stopwords')

from TurkishStemmer import TurkishStemmer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from os import path
from PIL import Image

from mlxtend.preprocessing import TransactionEncoder

import matplotlib.pyplot as plt
from collections import Counter

import pandas_profiling

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#Kaydettiğimiz csv dosyasını okuyoruz.
Twitterdf = pd.read_csv('KBTweet.csv',sep=';')

### Tekrar Edenlerin Temizlenmesi 

In [0]:
Twitterdf = Twitterdf.drop_duplicates(subset=['tweets'], keep=False)

### İndekslerin Yenilenmesi

duplike satırları sildiğimiz için indeksler de silindi, daha sonra indeksler ile çalışacağımız için indeksi yeniden başlatıyoruz.

In [0]:
Twitterdf.reset_index(drop=True, inplace=True)

In [0]:
# pandas_profiling nispeten yeni bir kütüphane, verilerin tümü hakkında tanımlayıcı istatistikleri bize okunabilir bir arayüz ile sunuyor
pandas_profiling.ProfileReport(Twitterdf, check_correlation = False)

Number of variables,6
Number of observations,2309
Total Missing (%),22.3%
Total size in memory,108.3 KiB
Average record size in memory,48.0 B
Numeric,1
Categorical,4
Boolean,0
Date,0
Text (Unique),1
Rejected,0


### Cevap Veren Hesapların Tweetlerinin Temizlenmesi

Bize kullanıcılar tarafından belediyeye gönderilen tweetler gerekli belediye tarafından kullanıcılara geri dönüş yapan tweetleri temizledik.

In [0]:
Twitterdf = Twitterdf.drop(Twitterdf[(Twitterdf.nick == 'kadikoybelediye') | (Twitterdf.nick == 'ibbBeyazmasa') ].index)

### Mention Temizleme

Tweetlerin içinde bize eğitimde zorluk yaşatacak olan link, mention gibi terimleri tweetlerin içinden temizledik. 


In [0]:
def mention_cleaner(text):
    stripped = re.sub(r"(?:\@|https?\://)\S+", "", text)
    return stripped

In [0]:
Twitterdf['tweets'] = Twitterdf['tweets'].apply(lambda x: mention_cleaner(x))
Twitterdf['tweets'].head()

0            tam "1" araç ile acimasizca müdahale e...
1    belediye  kurum   gunduz yaniyordu isiklar aks...
2      beyaz amerikan tirtil bocegi ama sadece turk...
3    her seferinde yüksekliği fazla olan bir kamyon...
4    evimin mükemmel manzarası 🤦🏻‍♀️  burası hasanp...
Name: tweets, dtype: object

### Emoji Temizleme

In [0]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [0]:
Twitterdf['tweets'] = Twitterdf['tweets'].apply(lambda x: deEmojify(x))

### Tüm Harflerin Küçültülmesi

eğitim için tüm karakterleri aynı formata dönüştürdük.

In [0]:
Twitterdf['tweets'] = Twitterdf['tweets'].str.lower()
Twitterdf['tweets'].head()

0            tam "1" ara ile acimasizca mdahale edi...
1    belediye  kurum   gunduz yaniyordu isiklar aks...
2      beyaz amerikan tirtil bocegi ama sadece turk...
3    her seferinde ykseklii fazla olan bir kamyon, ...
4    evimin mkemmel manzaras   buras hasanpaa hrriy...
Name: tweets, dtype: object

### Alfabe Temizleme

1. Bir alfabe seti tanımladık.
2. Metin içinde bu alfabe setinde uymayanları veri setinden temizledik.


In [0]:
alphabet = ('a','b','c','ç','d','e','f','g','ğ','h','i','ı','î','j','k','l','m','n','o','ö','p','q','r','s','ş','t','u','ü','v','w','x','y','z', ' ','û','ô','â','â')

In [0]:
def tweet_cleaner(text):
  for i in range(0,(len(text))):
    if not i : continue
    elif text[i] not in alphabet:
        text = text.replace(text[i],' ')
  return text

In [0]:
Twitterdf['tweets'] = Twitterdf['tweets'].apply(lambda x: tweet_cleaner(x) ) 
Twitterdf['tweets'].head()

0            tam     ara ile acimasizca mdahale edi...
1    belediye  kurum   gunduz yaniyordu isiklar aks...
2      beyaz amerikan tirtil bocegi ama sadece turk...
3    her seferinde ykseklii fazla olan bir kamyon  ...
4    evimin mkemmel manzaras   buras hasanpaa hrriy...
Name: tweets, dtype: object

### Stopwords Temizleme

Cümlelerin içinde çokça bağlaçlar ve zamirler bulunmaktadır. Bunlar özniteliklerimiz açısından bir anlam ifade etmemektedir. Stopwords fonksiyonu ile Türkçe bağlaç ve zamirler listesine ulaşabiliyoruz.

In [0]:
stop = set(stopwords.words('turkish'))

In [0]:
stop

{'acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'en',
 'eğer',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'ile',
 'ise',
 'için',
 'kez',
 'ki',
 'kim',
 'mu',
 'mü',
 'mı',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niye',
 'niçin',
 'o',
 'sanki',
 'siz',
 'tüm',
 've',
 'veya',
 'ya',
 'yani',
 'çok',
 'çünkü',
 'şey',
 'şu'}

In [0]:
# stopwords da eksik gördüğümüz kelimeleri,iyelik eklerini, bağlaçları v.s. update fonksiyonu ile ekledik.
stop.update(["mi","bir"])

### Kelime Bulutu 

Kelime bulutunda kullanmak üzere cümlelerimizi liste haline getirdik.

In [0]:
alltweet = " ".join(x for x in Twitterdf["tweets"])
alltweet

'        tam     ara ile acimasizca mdahale ediyor tirtillara oyle deme  ilacini buldular  yaklasik    sene sonra butun agaclar ilaclanacak  ama su kadikoy de kardiim  ne oyle her yer aa falan    cik cik cikksss    belediye  kurum   gunduz yaniyordu isiklar aksam oluyor kapkara    beyaz amerikan tirtil bocegi ama sadece turkiye de yok almanya ve hollanda da cok varmis orada haberlere bile cikmis her seferinde ykseklii fazla olan bir kamyon  i makinesi marmaray ayrlkeme orgeneral ahap grler caddesindeki st geide arpyor arkasndan gelsin trafik skan ambulanslar        evimin mkemmel manzaras   buras hasanpaa hrriyet sokak ya bir tane daha p konteyner  ekleyin ya da gn iinde almaya aln pencereyi dahi aamyoruz sinekten  ve kokudan   sayn belediye stma olcaz yldz rona ya bol dumanl bi sinek ilac arabas rica edicez  caddebostan migros un orada sahildeki ocuk oyun alanndaki platform zemini fazlasyla deforme olmu  haziran da da bildirmitik konuyu  ynlendirmesiyle  taklp dmek iin birebir  bu gib

In [0]:
wc = WordCloud(background_color="white", 
               max_words=2000, 
               min_font_size =12, 
               max_font_size=20, 
               relative_scaling = 0.5, 
               stopwords=stop,
               normalize_plurals= True)
wc.generate(alltweet)
plt.figure(figsize=(25,25))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

### Kelime Tekrarları

Elimdizeki kelimeleri ve ne kadar kullanıldıklarını gözlemledik.

In [0]:
word_list = alltweet.split()
countwords = Counter(word_list).most_common()
countwords

[('bir', 492),
 ('kadky', 413),
 ('bu', 375),
 ('ve', 362),
 ('de', 246),
 ('m', 233),
 ('in', 206),
 ('da', 200),
 ('i', 194),
 ('istanbul', 193),
 ('ne', 185),
 ('iin', 175),
 ('var', 175),
 ('at', 163),
 ('ok', 155),
 ('yok', 131),
 ('her', 127),
 ('sokak', 127),
 ('ama', 109),
 ('rt', 109),
 ('mi', 106),
 ('ile', 104),
 ('daha', 101),
 ('kadar', 100),
 ('o', 97),
 ('belediye', 94),
 ('caddesi', 93),
 ('gibi', 92),
 ('park', 85),
 ('ki', 84),
 ('deil', 83),
 ('trtl', 78),
 ('caddebostan', 75),
 ('ilalama', 75),
 ('ltfen', 70),
 ('ya', 69),
 ('su', 68),
 ('olarak', 68),
 ('neden', 66),
 ('merhaba', 66),
 ('badat', 65),
 ('hi', 61),
 ('diye', 58),
 ('ben', 58),
 ('zaman', 57),
 ('artk', 57),
 ('trtllar', 55),
 ('en', 55),
 ('ilgili', 52),
 ('yer', 50),
 ('iyi', 50),
 ('zm', 50),
 ('trkiye', 49),
 ('belediyesi', 48),
 ('oldu', 47),
 ('acaba', 47),
 ('rica', 45),
 ('ayn', 45),
 ('bile', 44),
 ('teekkrler', 44),
 ('olur', 44),
 ('acil', 44),
 ('sahili', 43),
 ('byle', 43),
 ('sevgili', 4

### Ek temizliği

Kelimeleri daha da temizlemek için eklerinden kurtulmamız gerekiyordu. Bu işlemi aşağıdaki kod ile gerçekleştirdik.

In [0]:
stemmer = TurkishStemmer()

Twitterdf['TweetClean'] = Twitterdf['tweets'].str.split()
Twitterdf['TweetClean'] = Twitterdf['TweetClean'].apply(lambda x: [stemmer.stem(y) for y in x])
Twitterdf['TweetClean'].head()

0    [tam, ar, il, acimasizca, mdahale, ediyor, tir...
1    [beledi, kur, gunt, yaniyor, isiklar, aksa, ol...
2    [beyaz, amerikan, tirtil, boceg, am, sade, tur...
3    [her, sefer, yksek, fazl, olan, bir, kamyon, i...
4    [ev, mkemmel, manzaras, buras, hasanpa, hrriye...
Name: TweetClean, dtype: object

Twitterdf'i bozmamak için playdf'e kopyaladık.

In [0]:
playdf = Twitterdf
playdf.reset_index(drop=True, inplace=True)

### Temizlenen Verinin Kaydedilmesi

In [0]:
Twitterdf.to_csv( 'KBTweetclean.csv',sep=';', index = False)

In [0]:
# 'Tweetclean' kolonunda kaç tane row olduğunu sayıyoruz
length = len(playdf['TweetClean'])

# df adında bir dataframe oluşturuyoruz
df = pd.DataFrame(columns = ['Words','Tweetid'])

In [0]:
#Association rule kullanarak kelimeler arasındaki ilşikileri görmek için kelimelerin herbiri bir satıra gelecek ve tweetid'leri ile 
# hangi cümle içinde kullanıldıkları ortaya çıkacak şekilde dataframe e yazdırıyoruz

for i in range(0, length-1):
  lentext = len(playdf['TweetClean'][i])

  for j in range(0,lentext-1):
    df1 = pd.DataFrame({'Words': playdf['TweetClean'][i][j],
                       'Tweetid': playdf['tweetid'][i]
                      } , index=[0] )
    
    df = df1.append(df)

### Association Rule ve Apriori

Association rule ile apriori algoritmasını kullanmak üzere ilgili kütüphaneleri yükledik.

In [0]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [0]:
te = TransactionEncoder()
te_ary = te.fit(playdf['TweetClean']).transform(playdf['TweetClean'])
df = pd.DataFrame(te_ary, columns=te.columns_)

### Apriori

Diğer kelimeler ile en çok hangi kelimelerin kullanıldığını ve bir arada hangi kelimelerin daha çok kullanıldığını gözlemledik.

In [0]:
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values(by='support',ascending= 0)

,support,itemsets
33,0.205078,(bir)
107,0.193359,(kadky)
89,0.183594,(i)
39,0.159668,(bu)
188,0.146484,(ve)
135,0.119629,(ne)
120,0.111328,(m)
51,0.110840,(de)
97,0.100098,(in)
100,0.099121,(istanbul)


### NGRAM

Ngram: Verilen bir dizilimdeki (sequence) tekrar oranını bulmaya yarayan yöntemdir. İsmi n ve gram kelimelerinin birleşiminden oluşmaktadır. Buradaki n, tekrarın kontrol edildiği değerdir. Gram ise bu tekrarın dizilim içerisindeki ağrılığını ifade etmek için kullanılmıştır.

In [0]:
def generate_ngrams(s, n):
    tokens = [token for token in s.split(" ") if token != ""]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [0]:
generate_ngrams(alltweet, 3)

['tam ara ile',
 'ara ile acimasizca',
 'ile acimasizca mdahale',
 'acimasizca mdahale ediyor',
 'mdahale ediyor tirtillara',
 'ediyor tirtillara oyle',
 'tirtillara oyle deme',
 'oyle deme ilacini',
 'deme ilacini buldular',
 'ilacini buldular yaklasik',
 'buldular yaklasik sene',
 'yaklasik sene sonra',
 'sene sonra butun',
 'sonra butun agaclar',
 'butun agaclar ilaclanacak',
 'agaclar ilaclanacak ama',
 'ilaclanacak ama su',
 'ama su kadikoy',
 'su kadikoy de',
 'kadikoy de kardiim',
 'de kardiim ne',
 'kardiim ne oyle',
 'ne oyle her',
 'oyle her yer',
 'her yer aa',
 'yer aa falan',
 'aa falan cik',
 'falan cik cik',
 'cik cik cikksss',
 'cik cikksss belediye',
 'cikksss belediye kurum',
 'belediye kurum gunduz',
 'kurum gunduz yaniyordu',
 'gunduz yaniyordu isiklar',
 'yaniyordu isiklar aksam',
 'isiklar aksam oluyor',
 'aksam oluyor kapkara',
 'oluyor kapkara beyaz',
 'kapkara beyaz amerikan',
 'beyaz amerikan tirtil',
 'amerikan tirtil bocegi',
 'tirtil bocegi ama',
 'bocegi a

# **3. Adım**:  **Makine Öğretisi**

---

Makine öğetisine başlamadan önce elimizde bulunan bütün tweetleri okuduk ve bunları daha önceden belirlediğimiz kategorilerine (Bilgi, Talep, Selemlama, Diğer) göre işaretledik.

### Etiketlenen Dosyanın Okunması

In [0]:
Twitterdf = pd.read_csv('tweet.csv',sep=',')

### Label Encoding İşlemi

Label Encoder: Etiketleri makinede okunabilir forma dönüştürmek için sayısal forma dönüştürme anlamına gelir. Makine öğrenme algoritmaları bu etiketlerin nasıl kullanılması gerektiğine daha iyi bir şekilde karar verebilir. Denetimli öğrenmede yapılandırılmış veri kümesi için önemli bir ön işlem adımıdır.

In [0]:
from sklearn import preprocessing 
import numpy as np   
label_encoder = preprocessing.LabelEncoder() 
Twitterdf['tags'] = label_encoder.fit_transform(Twitterdf['tags']) 
Twitterdf['tags'].unique()

array([0, 3, 2, 1])

### Verinin Bölünmesi

Veriyi kurduğumuz modeli test etmek için belirledimiz oranda böldük. Biz bu oranı %20 test %80 train olarak belirledik.

In [0]:
from sklearn.model_selection import train_test_split
y = Twitterdf['tags']
sentences = Twitterdf['tweets']
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.20, stratify = y, random_state=42)
print(sentences_train.shape)
print(sentences_test.shape)

(1441,)
(361,)


### TF-IDF Vektörize İşlemi

TF-IDF:  kavramı IR (information retrieval, bilgi getirimi) gibi konuların altında bir sıralama (ranking) algoritması olarak sıkça geçmektedir. İngilizcedeki Term Frequency – Inverse Document Frequency (Terim frekansı – ters metin frekansı) olarak geçen kelimelerin baş harflerinden oluşan terim basitçe bir metinde geçen terimlerin çıkarılması ve bu terimlerin geçtiği miktara göre çeşitli hesapların yapılması üzerine kuruludur. Klasik olarak TF yani terimlerin kaç kere geçtiğinden daha iyi sonuç verir. Kısaca TF-IDF hesabı sırasında iki kritik sayı bulunmaktadır. Bunlardan birincisi o anda ele alınan dokümandaki terimin sayısı diğeri ise bu terimi külliyatta içeren toplam doküman sayısıdır.

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(Twitterdf['tweets'])
xtrain_tfidf =  tfidf_vect.transform(sentences_train)
xvalid_tfidf =  tfidf_vect.transform(sentences_test)
print(xtrain_tfidf.shape)
print(xvalid_tfidf.shape)

(1441, 5000)
(361, 5000)


In [0]:
X_train = xtrain_tfidf
X_test = xvalid_tfidf

### İlk Modelimiz Logistic Regression

Lojistik regresyon, yalnızca iki değere sahip olabilen bir sonucun olasılığını öngörür (yani, ikiye bölünebilir). Tahmin, bir veya birkaç öngörücünün (sayısal ve kategorik) kullanımına dayanır. Doğrusal regresyon evet/hayır, var/yok gibi binary(ikili) sistemde ifade edilebilecek değerler için uygun değildir. Çünkü,0 ve 1 aralığının dışında değer tahmin edebilir.

Lojistik regresyon, 0 ile 1 arasındaki değerlerle sınırlı lojistik eğrisi üretir. Lojistik regresyon lineer bir regresyona benzer, ancak eğri olasılık yerine hedef değişkenin olasılıkları’ nın doğal logaritması kullanılarak oluşturulur.

### GridSearchCV

Model'e en uygun parametreler belirlenmesiiçin GridSearchCV metodunu kullandık. Bu metod tek tek belirlediğiniz parametrelerin belirlediğiniz aralıklarını deneyerek en iyi sonuç veren parametreleri belirliyor.

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

grid={"C":np.logspace(0.0001,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

y_pred = logreg_cv.predict(X_test)
a = mean_squared_error(y_test, y_pred)

train_score = logreg_cv.score(X_train, y_train)
test_score = logreg_cv.score(X_test, y_test)
f1 = f1_score(y_test, y_pred, average='macro')
print('mse',a)
print("Train Accuracy:", train_score)
print("Test Accuracy:", test_score)
print("F1:", f1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tuned hpyerparameters :(best parameters)  {'C': 100.00767557820069, 'penalty': 'l2'}
accuracy : 0.6023594725884802
mse 1.734072022160665
Train Accuracy: 0.9993060374739764
Test Accuracy: 0.5429362880886427
F1: 0.5746823965392628


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### İkinci Modelimiz Naive Bayes

Naive Bayes sınıflandırıcısının temeli Bayes teoremine dayanır. lazy ( tembel ) bir öğrenme algoritmasıdır aynı zamanda dengesiz veri kümelerinde de çalışabilir. Algoritmanın çalışma şekli bir eleman için her durumun olasılığını hesaplar ve olasılık değeri en yüksek olana göre sınıflandırır. Az bir eğitim verisiyle çok başarılı işler çıkartabilir. Test kümesindeki bir değerin eğitim kümesinde gözlemlenemeyen bir değeri varsa olasılık değeri olarak 0 verir yani tahmin yapamaz. Bu durum genellikle Zero Frequency ( Sıfır Frekans ) adıyla bilinir. Bu durumu çözmek için düzeltme teknikleri kullanılabilir. En basit düzeltme tekniklerinden biri Laplace tahmini olarak bilinir.

In [0]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB

nb=BernoulliNB(alpha=0.0001,binarize=0.0,fit_prior=True,class_prior =None)
nb_model=nb.fit(X_train,y_train.ravel())

# Test veri kümemizi verdik ve iris türü tahmin etmesini sağladık
result = nb_model.predict(X_test)

# Başarı Oranı
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, result)
# Sonuç : 0.96
print('train accuracy', nb_model.score(X_train,y_train))
print('test accuracy', accuracy)

train accuracy 0.9791811242192922
test accuracy 0.5096952908587258


# **4. Adım**: **Derin Öğrenme**

---


### Sequential Modelinin Oluşturulması

Sequential, Keras’ta bir model oluşturmanın en kolay yoludur. Katman ile bir model katman oluşturmanıza izin verir. Her katmanın aşağıdaki katmanı gösteren ağırlıklara sahiptir.

In [0]:
from keras.models import Sequential
from keras import layers
from keras import optimizers
from keras.regularizers import l2

input_dim = X_train.shape[1]  # Number of features
cat_num = len(y.unique()) # Number of categories

model = Sequential()
model.add(layers.Dense(64, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(64,kernel_regularizer=l2(0.001), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64,kernel_regularizer=l2(0.001), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64,kernel_regularizer=l2(0.001), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64,kernel_regularizer=l2(0.001), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(cat_num, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                320064    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      

In [0]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

classNum = len(np.unique(y_train)) # number of unique labels is counted to determine the # of classes
y_train = LabelEncoder().fit_transform(y_train)
y_train = to_categorical(y_train, num_classes = classNum)
y_test = LabelEncoder().fit_transform(y_test)
y_test = to_categorical(y_test, num_classes = classNum)

### Derin Öğrenme Eğitimi

In [0]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=True,
                    batch_size=512,
                    validation_data=(X_test, y_test)
                    )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1441 samples, validate on 361 samples
Epoch 1/50
1441/1441 [==============================] - 1s 810us/step - loss: 1.6430 - acc: 0.2491 - val_loss: 1.6352 - val_acc: 0.2936
Epoch 2/50
1441/1441 [==============================] - 0s 84us/step - loss: 1.6316 - acc: 0.2720 - val_loss: 1.6237 - val_acc: 0.2825
Epoch 3/50
1441/1441 [==============================] - 0s 95us/step - loss: 1.6206 - acc: 0.2804 - val_loss: 1.6123 - val_acc: 0.3075
Epoch 4/50
1441/1441 [==============================] - 0s 79us/step - loss: 1.6086 - acc: 0.2700 - val_loss: 1.6009 - val_acc: 0.3102
Epoch 5/50
1441/1441 [==============================] - 0s 81us/step - loss: 1.5974 - acc: 0.2811 - val_loss: 1.5897 - val_acc: 0.3047
Epoch 6/50
1441/1441 [==============================] - 0s 79us/step - loss: 1.5856 - acc: 0.2929 - val_loss: 1.5786 - val_acc: 0.2909
Epoch 7/50
1441/1441 [==========================

### Başarı Sonuçları

In [0]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9986
Testing Accuracy:  0.5374


### Çıkan Sonuçların Görselleştirilmesi

In [0]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    
    
plot_history(history)

# **Sonuç**

---

Teknolojinin gelişmesi ile birlikte, Yerel Yönetimlerin sunduğu hizmet modellerini desteklemek ve güçlendirmek mümündür. 

Sosyal Medya birey – kurum etkileşimi açısından, doğrudan demokrasiye yaklaşıması açısından, çok önemli bir imkan sunmaktadır. Yerel yönetimlerin mali ve mevzuat kısıtları yüzünden; planlasa da; gerçekleştiremediği demokratik modelleri, Yapay Zeka kullanarak gerçekleştirmek her geçen gün daha da mümkün olmaktadır. 

Atölye kapsamında gerçekleştirdiğimiz çalışma, twitlere nasıl erişilebildiğini, ne şekilde analiz edildiğini ve bu analizler eşliğinde, problemlere hangi yapay zeka algoritmaları ile müdahale edilebileceğini göstermektedir. 

Çalışma ile amacımız bir yandan Yerel Yönetimler için, Bilgisayar Mühendislerinin, nasıl katkı sunabileceğini ve hangi teknikleri kullanılabileceğini öğrenmek; diğer yandan Kadıköy Belediyesine; Yapay Zeka kullanarak hangi problemlerine, ne şekilde çözümler ile yaklaşılabileceğini göstermek oldu. 

### Model Sonuçları

Kurduğumuz modellerde elde ettiğimiz sonuçlar düşük gözükse bile aslında iyi denebilecek sonuçlar.

### Sonuçların Nedenleri

Sonuçların %50 civarında olmasının nedeni aslında ele alınan veri setinin hem boyut hem de çeşitlilik olarak yetersizliğiydi. Tweetlerin gündeminin belirli konular içerisinde kalması başarımıza etki etti. 

### Başarı Nasıl Artar?

Başarıyı artırmamızın en gerekli ihtiyacı veri setinin büyüklüğü ve bu veri setinde ki veri çeşitliliğinin miktarı. Bir yıl boyunca toplanacak olan tweetler ile başarıdaki sonuç çok daha yüksek bir oranda olacaktır.

### Gelecek Çalışmalar 

Çalışmamız bir atölye çalışması olup, öğrenme odaklı olarak zaman ve veri kısıtı ile gerçekleştirilmiştir. Bu amaçla twitler için belirlenen kategoriler sınırlı sayıda seçilmiştir; Bununla birlikte modeller için parametre değişiklikleri ve ek veri sağlanması halinde modellerin başarı oranlarının artması mümkündür. 

Çalışma aynı zamanda “Kadıköy” ve “Kadıköy Belediyesi” terimlerine odaklanmış olup, Belediyeye ait farklı unsurlar üzerinde çalışmamıştır. Örneğin projenin genişletilmesi halinde Belediye başkanın hakkındaki görüşleride analiz etmek mümkündür. 

Çalışmanın sürdürülmesi halinde kadıköy belediyesi ile ilgili tüm twitlere cevap olma fırsatı bir gerçeklik olarak durmaktadır. Bununla birlikte belediyenin demokratik yönetişim kavramları içinde halktan, belediyenin politikaları ,belediyenin çalışmaları, belediye başkanın çalışmaları hakkında etkileşimde bulunulması ve çağın gerekliliklerine göre bir yönetim modeli oluşturulması mümkündür. 

Yerel Yönetimlerin stratejik plan hazırladığı dönemin içinde olduğumuz göz önünde bulundurulursa; Kadıköy Belediyesinin belirlediği amaç ve hedefler hakkında, Kadıköy halkının düşüncelerinin olumlu olup olmadığını belirlemek, çalışma süreçlerinin modifiye edilmesi ile mümkün olacaktır. 